In [1]:
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
#train_path = r'C:\Users\user\Desktop\FaceDetection_Using_VGG16\train_data\\'
#valid_path = r'C:\Users\user\Desktop\FaceDetection_Using_VGG16\test_data\\'
import os
base_dir = 'Dataset/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# directory with our training cat/dog pictures

train_p1_dir = os.path.join(train_dir, 'Harry')
#train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat/dog pictures

validation_p1_dir = os.path.join(validation_dir, 'Harry')
#validation_dogs_dir = os.path.join(validation_dir, 'dogs')


In [3]:
print("number of Images in p1 train dir      : ", len(os.listdir(train_p1_dir)))
print('total validation images in p1 dir :', len(os.listdir( validation_p1_dir ) ))

number of Images in p1 train dir      :  100
total validation images in p1 dir : 100


In [4]:

model = VGG16(input_shape = (224,224,3), weights ='imagenet', include_top =False)

In [5]:
for layer in model.layers:
    layer.trainable = False

In [6]:
x = Flatten()(model.output)

In [7]:
prediction = Dense(1, activation = 'sigmoid')(x)

In [8]:
new_model = Model(inputs = model.input, outputs = prediction)

In [9]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [10]:
new_model.compile(
        loss= 'binary_crossentropy',
        optimizer = 'adam',
        metrics = ['acc']
        )

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [12]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 10,
                                                    class_mode = 'binary', 
                                                    target_size = (224, 224))

Found 100 images belonging to 1 classes.


In [13]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)  #instance of imagedatagenerator for validation data

validation_generator = test_datagen.flow_from_directory(

                        validation_dir,
                        target_size = (224,224),
                        batch_size = 10,
                        class_mode = 'binary'
)

Found 100 images belonging to 1 classes.


In [14]:
history = new_model.fit_generator(

                    train_generator,
                    steps_per_epoch = 20,        
                    epochs = 5,
                    validation_data = validation_generator,
                    validation_steps = 15,
                    )

Epoch 1/5
20/20 [==============================] - 35s 2s/step - loss: 0.0468 - acc: 0.9500 - val_loss: 2.7418e-09 - val_acc: 1.0000
Epoch 2/5
20/20 [==============================] - 25s 1s/step - loss: 3.5945e-09 - acc: 1.0000 - val_loss: 2.8790e-10 - val_acc: 1.0000
Epoch 3/5
20/20 [==============================] - 25s 1s/step - loss: 2.2363e-09 - acc: 1.0000 - val_loss: 1.3243e-10 - val_acc: 1.0000
Epoch 4/5
20/20 [==============================] - 25s 1s/step - loss: 1.6322e-09 - acc: 1.0000 - val_loss: 2.0326e-10 - val_acc: 1.0000
Epoch 5/5
20/20 [==============================] - 25s 1s/step - loss: 2.4907e-09 - acc: 1.0000 - val_loss: 2.4847e-10 - val_acc: 1.0000


In [15]:
from keras.models import load_model

In [16]:
model.save("Face_Detection_vgg.h5")